In [1]:
from Classifier.load_data import read_arduino, process_data, read_arduinbro, load_training_data
from Classifier.classifier import classify_event, zeroes_classifier, one_pronged_smoothing_classifier, streaming_classifier

from copy import deepcopy
from Levenshtein import distance as levenshtein_distance
from scipy import signal
import serial
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from scipy.io import wavfile

In [55]:
"""
from numba import njit

@njit
"""
def get_event_regions(wav_array, samprate, labels_dat, before_buffer = 0.7,
                       after_buffer = 1):
    
    time_seq = np.linspace(1, len(wav_array), len(wav_array))/samprate

    

    left_events_bool = np.array([False]*len(time_seq))
    for time in labels_dat.time[labels_dat.label == "L"]:
        left_events_bool = ( (time_seq > time - before_buffer) & (time_seq < time+after_buffer) ) | left_events_bool

    right_events_bool = np.array([False]*len(time_seq))
    for time in labels_dat.time[labels_dat.label == "R"]:
        right_events_bool = ( (time_seq > time - before_buffer) & (time_seq < time+after_buffer) ) | right_events_bool

    event_bool = np.bitwise_or(left_events_bool, right_events_bool)
    return event_bool

#@njit
def contrast_all_files(window_size, test_stat_fn,
                samprate,       #not important
                waves,     #not important
                labels):   #not important
    start = time.time()
    event_test_stats = np.array([])
    non_event_test_stats = np.array([])
    for key in waves.keys():
        wav_array = waves[key]
        labels_dat = labels[key]
        
        total_time = len(wav_array)/samprate
        
        events_bool = get_event_regions(wav_array, samprate, labels_dat)
        
        index = 0
        step = int(0.1*samprate)
        test_stats = np.array([None]*len(wav_array))
    
        while index + 1 < len(wav_array):
            subset = wav_array[index:(index+window_size)]
            test_stats[index:index+step] = test_stat_fn(subset)
            index += step
        #print("test_stats", test_stats[events_bool])
        #print("len test stats: ", len(test_stats[events_bool]) + len(test_stats[~events_bool]))
        #print("event_test_stats", event_test_stats)
        event_test_stats = np.append(event_test_stats, test_stats[events_bool])
        non_event_test_stats = np.append(non_event_test_stats, test_stats[~events_bool])
    #need to fix appends ^
    if len(event_test_stats) > 0 and len(non_event_test_stats) > 0:
        contrast_stat = (np.amin(event_test_stats) - np.amax(non_event_test_stats))
        print(contrast_stat)
        end = time.time()
        print("time taken: ", end - start)
        return contrast_stat
    else:
        end = time.time()
        print("time taken: ", end - start)
        print("len 0!!!!")

In [56]:
"""from numba import njit

@njit"""
def range_test_stat(subset):
    return np.max(subset) - np.min(subset)

In [57]:
waves, labels, samprate = load_training_data(path = "/Users/darap/Documents/School/University/2021, Sem. 1/DATA3888/Aqua10/Datasets/Good Data - Sandeep no errors/",
                       scale_factor= 512/(2**13 - 1),
                       blacklist = ["blink", "different", "fast", "slow", "eyebrow"])
"""
labels1 = np.array([])
waves1 = []
i = 0
for key in waves.keys():
    waves1.append(list(waves[key]))
print(type(waves1))
print(type(waves1[1]))
print(type(waves1[1][1]))
waves1 = np.ndarray(waves1, np.float64)
print(waves1[1])
print(len(waves1))
print(len(waves1[5]))
"""
contrasts = []
for x in range(1, 20000, 100):
    contrasts.append(contrast_all_files(window_size = x, test_stat_fn = range_test_stat,
                    samprate = samprate,
                    waves = waves,
                    labels = labels))


c:\Users\darap\Documents\School\University\2021, Sem. 1\DATA3888\Aqua10\Classifier\load_data.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  labels_dat = pd.read_csv(path+file, sep=",\t", skiprows=1)
dict_keys(['left-middle-right-middle#2', 'left-middle-right-middle', 'left-middle-right-steph', 'left-middle-right-steph2', 'left-middle', 'left-right-middle-marina', 'left-right-middle-marina2', 'left-right-middle-marina3', 'left-right-middle-sandeep', 'right-middle']) dict_keys(['left-middle-right-middle#2', 'left-middle-right-middle', 'left-middle-right-steph', 'left-middle-right-steph2', 'left-middle', 'left-right-middle-marina', 'left-right-middle-marina2', 'left-right-middle-marina3', 'left-right-middle-sandeep', 'right-middle'])
0.0
time taken:  4.548992872238159
-162.20730069588572
tim

KeyboardInterrupt: 

In [39]:
a = [[1,2,3], [4,5,6]]
b = np.ndarray()

[1, 2, 3, 4, 5, 6]
